In [37]:
# let user input dataset name

import pandas as pd
import os
import seaborn as sns
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
dataset_name = input("Enter dataset name: ")

# Search file with levinstein distance

dataset_files = []
for root, dirs, files in os.walk("data"):
    for file in files:
        if file.endswith(".csv"):
            dataset_files.append(file)


def calculate_levenshtein(str1, str2):
    replacementCost = [[]]

    for i in range(len(str1) + 1):
        replacementCost.append([])
        for j in range(len(str2) + 1):
            if i == 0:
                replacementCost[i].append(j)
            elif j == 0:
                replacementCost[i].append(i)
            elif str1[i - 1] == str2[j - 1]:
                replacementCost[i].append(replacementCost[i - 1][j - 1])
            else:
                replacementCost[i].append(1 + min(replacementCost[i - 1][j], replacementCost[i][j - 1],
                                                  replacementCost[i - 1][j - 1]))

    return replacementCost[len(str1)][len(str2)]


# Find the closest file
scores = []

for file in dataset_files:
    print(file)
    scores.append(calculate_levenshtein(dataset_name, file.split("_data")[0]))

closest_file_name = dataset_files[scores.index(min(scores))]
print("Closest file: " + closest_file_name)

# Load the dataset
df = pd.read_csv("data/" + closest_file_name)

df = df.drop(columns=["imdb_id", "id"])

# print(df)
df["score_avg"] = (df["imdb_score"] + df["tmdb_score"]) / 2

hulu-tv_data.csv
disney-+_data.csv
amazon-prime_data.csv
rakuten-viki-tv_data.csv
netflix_data.csv
paramount-tv_data.csv
dark-matter-tv_data.csv
crunchyroll_data.csv
hbo-max_data.csv
Closest file: netflix_data.csv


In [38]:
# get unique genres
df["genres_encoded"] = df["genres"].astype("category").cat.codes
df[["genres", "genres_encoded"]].astype("category")


,genres,genres_encoded
0,['documentation'],624
1,"['crime', 'drama']",533
2,"['comedy', 'fantasy']",416
3,['comedy'],473
4,['horror'],1116
...,...,...
5801,"['romance', 'drama']",1200
5802,"['music', 'documentation']",1122
5803,"['family', 'drama']",945
5804,"['action', 'thriller']",171


In [39]:
# Split in train and test set
X = df.drop(columns=['score_avg', 'title'])
y = df['score_avg']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

score = accuracy_score(y_test, predictions)
score


ValueError: could not convert string to float: 'MOVIE'